# PHASE 0 - DATA INTEGRATION

## 0.1 - Data Quality Check

In [39]:
import sys
import os

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

from src.data_utils \
import load_orders, load_products, \
load_order_products_prior, load_order_products_train, \
load_departments, load_aisles

In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

SEED = 000

In [42]:
# -----------------------------------------------
# Load 6 csv
# -----------------------------------------------

orders = load_orders()
products = load_products()
order_products_prior = load_order_products_prior()
order_products_train = load_order_products_train()
departments = load_departments()
aisles = load_aisles()

In [43]:
print(f"Orders shape: {orders.shape}\n")
display(orders.head())

Orders shape: (3421083, 7)



,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [44]:
print(f"Orders shape: {products.shape}\n")
display(products.head())

Orders shape: (49688, 4)



,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [45]:
# -----------------------------------------------
# Check prior and train overlap
# -----------------------------------------------

# Check if train is a subset of prior
train_orders = set(order_products_train['order_id'].unique())
prior_orders = set(order_products_prior['order_id'].unique())

# Are they mutually exclusive or overlapping?
overlap = train_orders & prior_orders
print(f"Overlapping orders: {len(overlap)}")
print(f"Train-only orders: {len(train_orders - prior_orders)}")
print(f"Prior-only orders: {len(prior_orders - train_orders)}")

Overlapping orders: 0
Train-only orders: 131209
Prior-only orders: 3214874


In [53]:
train_plus_prior = len(train_orders - prior_orders) + len(prior_orders - train_orders)
print(f"order_id counts in order_products tables: {train_plus_prior}")

order_id counts in order_products tables: 3346083


In [54]:
print(f"order_id counts not available in order_products tables: {orders.shape[0] - train_plus_prior}")

order_id counts not available in order_products tables: 75000


In [64]:
# -----------------------------------------------
# Remove Kaggle's test set
# -----------------------------------------------

# Check eval_set distribution
print("Orders by eval_set:")
print(orders['eval_set'].value_counts())
print(f"\nTotal orders: {len(orders)}")

# Filter to keep only 'prior' and 'train'
orders_filtered = orders[orders['eval_set'].isin(['prior', 'train'])].copy()

print(f"\nOrders after filtering: {len(orders_filtered)}")
print(f"Removed: {len(orders) - len(orders_filtered)}")

# Verify all orders now are consistent with order_products data
orders_without_products = set(orders_filtered['order_id'].unique())
products_orders = set(order_products_prior['order_id'].unique()) | set(order_products_train['order_id'].unique())

print(f"\nOrders without product data: {len(orders_without_products)}")
print(f"Missing: {len(orders_without_products - products_orders)}")  # Should be 0

# Replace original orders
orders = orders_filtered

Orders by eval_set:
eval_set
prior    3214874
train     131209
test       75000
Name: count, dtype: int64

Total orders: 3421083

Orders after filtering: 3346083
Removed: 75000

Orders without product data: 3346083
Missing: 0
